In [46]:
import os
from django.contrib.gis.gdal import DataSource
from django.db.models.signals import post_save, post_delete
from mpa.models import mpa_post_save

# restoregeo = os.path.abspath('/Users/russmo/Code/postgres/mpatlas_20191231_georestore.gpkg')
cea_updates = os.path.abspath('/Users/russmo/Code/postgres/CEA_countries_edited_20191204.gdb')
# kawesqar = os.path.abspath('/Users/russmo/Code/postgres/Kawesqar_simple_20191204.gdb')
recents = os.path.abspath('/Users/russmo/Code/postgres/mpatlas_20190130_ABNJ_MOZ_edits.gpkg')
# recents = os.path.abspath('/Users/russmo/Code/postgres/mpatlas_20200127_local_point.gpkg')


In [47]:
ds = DataSource(recents)
print(ds)
print(len(ds))

/Users/russmo/Code/postgres/mpatlas_20190130_ABNJ_MOZ_edits.gpkg (GPKG)
1


In [48]:
mpa_poly = ds[0]
for lyr in ds:
    print (lyr, lyr.geom_type, len(lyr), lyr.srs,)
    f_types = [fld.__name__ for fld in lyr.field_types]
#     print (list(zip(lyr.fields, f_types, lyr.field_widths)))

mpatlas_20190130_ABNJ_MOZ_edits MultiPolygon 12 GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]


In [49]:
from mpa.models import Mpa
src = mpa_poly
print('Importing', len(src), 'records from MPA edit table')

Importing 12 records from MPA edit table


In [36]:
allfields = src.fields.copy()
removefields = [
    'mpa_id',
    # 'summary',
    # 'fishing_info',
    'geom_Length',
    'geom_Area',
    'status_year_nonull',
    'Shape_Length',
    'Shape_Area',
    'categories'
]
fields = [f for f in allfields if f not in removefields]

In [50]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
for feat in src:
    mid = feat['mpa_id'].value
    if mid == 68807533: continue
    if mid in (
        68813349, 68813344, 68813348, 68813347, 68813341, 68813342, 68813343, 68813345, 68813346, 68813350, 68813351
    ):
        print('!!', mid, feat, feat['mpa_id'], feat['name'])
        # m = Mpa.objects.get(mpa_id=mid)
        m = Mpa(name=feat['name'].value)
        for f in fields:
            setattr(m, f, feat[f].value)
        m.geom = feat.geom.geos
        m.save()
        print(feat.fid, mid, flush=True)
        # print(feat.fid, mid, end='\r', flush=True)
# post_save.connect(mpa_post_save, sender=Mpa)

!! 68813351 Feature FID 1 in Layer<mpatlas_20190130_ABNJ_MOZ_edits> 68813351 Western Antarctic Peninsula/Domain 1 - Krill Fishery Zone
Carto Error for mpa_id 68817245: Access denied
1 68813351
!! 68813350 Feature FID 2 in Layer<mpatlas_20190130_ABNJ_MOZ_edits> 68813350 Western Antarctic Peninsula/Domain 1 - General Protection Zone
Carto Error for mpa_id 68817246: Access denied
2 68813350
!! 68813349 Feature FID 3 in Layer<mpatlas_20190130_ABNJ_MOZ_edits> 68813349 Mandzenika
Carto Error for mpa_id 68817247: Access denied
3 68813349
!! 68813348 Feature FID 4 in Layer<mpatlas_20190130_ABNJ_MOZ_edits> 68813348 Nha Dzi Sectori
Carto Error for mpa_id 68817248: Access denied
4 68813348
!! 68813347 Feature FID 5 in Layer<mpatlas_20190130_ABNJ_MOZ_edits> 68813347 Mahigo Mbate
Carto Error for mpa_id 68817249: Access denied
5 68813347
!! 68813346 Feature FID 6 in Layer<mpatlas_20190130_ABNJ_MOZ_edits> 68813346 Marrambone
Carto Error for mpa_id 68817250: Access denied
6 68813346
!! 68813345 Featur

In [90]:
badpts = mpas_all_nogeom.filter(geom__isnull=True, is_point=False, point_geom__isnull=True)
badpts.count()
for b in badpts:
    print(b.mpa_id, b.wdpa_id, b.country, b.name, b.designation)

68817244 555624907 COK Cook Islands Marine Park
 Marine Park
60009494 None KNA S. E. Peninsula Marine Protected Area 
60009484 None KNA Bath Bogs 
68807590 None CUB Delta del Mayari 
9379 None DEU Roter Sand Nature Reserve
60009585 None DOM Playa de Andres Boca Chica Ecological Corridor
60009599 None JAM Peak Bay Forest Reserve
68807582 None CUB Maisφ - Caleta 
60009485 None VCT South Coast Marine Conservation Area Marine Conservation Area
60009483 None VCT King's Hill Forest Reserve Forest Reserve
60009468 None NLD Hermanus National Park
60009520 None DOM Boca de Barracote-Punta Los Cocos No Take Area
60009470 None TTO St. David Reserve Nature Reserve
60009521 None DOM Arroyo La Ceja y Cano El Pinito No Take Area
60009478 None TTO Nrra. Nature Reserve
60009596 None DOM Rio Cumayasa Cuevas Las Maravillas Monumento Natural
60009553 None GRD Conference Bay Priority Area of Interest
60009530 None JAM Galleon, St Elizabeth Special Fishery Conservation Area Special Fishery Conservation Area

68807737 None GBR Minsmere-Walberswick Special Protection Area (Birds Directive)
68808411 None KHM Cambodia Bottom Trawl Closure
60009538 None HTI Complexe Cap-Camp Louise 
60009566 None HTI Abacou 
60009525 None HTI Cote Belle Anse et Grand Gosier 
60009580 None HTI Dame Marie 
60009578 None HTI La Cahoune 
60009543 None HTI La Navase 
60009524 None HTI Petit Paradis 
68808424 None PAK Indus River Canyon Marine Protected Area
60009526 None HTI Gonaives-Saint Marc Parc Marin
68808087 None NZL Ōhau Point - Fur Seal Sanctuary
68808341 None HTI Jeremie-Abricot Parc Marin
68808313 None NZL Otago Shelf and Canyons Marine Protected Area
68808462 None CAN Pingo Canadian Landmark
68807887 None BRA Tupinambas Estação Ecológica
68808332 None LKA Kuda Ravana Maha Ravana Marine Sanctuary
68808331 None LKA Kayankarni Marine Natural Reservation
68813328 None LKA Wedithalathivu Nature Reserve
68808336 None AUS Cape Byron Headland Reserve State Conservation Area
68808335 None AUS Arakwal National Park

In [45]:
feats = []
for feat in src:
    feats.append(feat['mpa_id'].value)

[68813470,
 68813469,
 68813468,
 68813467,
 68813466,
 68813465,
 68813464,
 68813463,
 68813462,
 68813461,
 68813460,
 68813459,
 68813458,
 68813457,
 68813456,
 68813455,
 68813454,
 68813453,
 68813452,
 68813451,
 68813450,
 68813449,
 68813448,
 68813447,
 68813446,
 68813445,
 68813444,
 68813443,
 68813442,
 68813441,
 68813440,
 68813439,
 68813438,
 68813437,
 68813436,
 68813435,
 68813434,
 68813433,
 68813432,
 68813431,
 68813430,
 68813429,
 68813428,
 68813427,
 68813426,
 68813425,
 68813424,
 68813423,
 68813422,
 68813421,
 68813420,
 68813419,
 68813418,
 68813417,
 68813416,
 68813415,
 68813414,
 68813413,
 68813412,
 68813411,
 68813410,
 68813409,
 68813408,
 68813407,
 68813406,
 68813405,
 68813404,
 68813403,
 68813402,
 68813401,
 68813400,
 68813399,
 68813398,
 68813397,
 68813396,
 68813395,
 68813394,
 68813393,
 68813392,
 68813391,
 68813390,
 68813389,
 68813388,
 68813387,
 68813386,
 68813385,
 68813384,
 68813383,
 68813382,
 68813381,
 68813380,

In [163]:
import urllib.request, json 
for feat in src:
    mid = feat['mpa_id'].value
    if len(feat['summary'].value) >= 255:
        print(feat['name'].value, "http://www.mpatlas.org/mpa/sites/%s/json/" % (mid))
        try:
            with urllib.request.urlopen("http://www.mpatlas.org/mpa/sites/%s/json/" % (mid)) as url:
                data = json.loads(url.read().decode())
                print(mid, len(data['summary']))
                m = Mpa.objects.filter(mpa_id=mid)
                print(len(m.first().summary))
                # m.update(summary=data['summary'])
        except(urllib.request.HTTPError):
            print(mid, 'Not on MPAtlas production server')
            m = Mpa.objects.filter(mpa_id=mid)
            print(len(m.first().summary))

Tierra Blanca http://www.mpatlas.org/mpa/sites/68808633/json/
68808633 Not on MPAtlas production server
3685
Lobería Selkirk http://www.mpatlas.org/mpa/sites/68808632/json/
68808632 Not on MPAtlas production server
3685
El Arenal http://www.mpatlas.org/mpa/sites/68808631/json/
68808631 Not on MPAtlas production server
3685
Montes submarinos Crusoe y Selkirk http://www.mpatlas.org/mpa/sites/68808630/json/
68808630 Not on MPAtlas production server
3685
Mar de Juan Fernández http://www.mpatlas.org/mpa/sites/68808392/json/
68808392 3685
3685
Nazca-Desventuradas http://www.mpatlas.org/mpa/sites/9175/json/
9175 5767
5767
Pacifico Mexicano Profundo - Core Zones http://www.mpatlas.org/mpa/sites/68808479/json/
68808479 6306
6306
Juan Fernández http://www.mpatlas.org/mpa/sites/68808467/json/
68808467 5024
5024
Mar de Cabo de Hornos (Diego Ramírez) http://www.mpatlas.org/mpa/sites/68808466/json/
68808466 4249
4249
Archipiélago de Revillagigedo http://www.mpatlas.org/mpa/sites/68808404/json/
68808

In [161]:
mods = '''
68808485 BRA BRA !NEW! Naufrágio Queimado Parque Estadual Marinho 2018-10-12 12:20:53.423198
68807800 GBR GBR Flamborough Head Special Areas of Conservation (Habitats Directive) 2018-10-08 13:52:33.050569
68808483 GBR GBR !NEW! Flamborough Head [No-Take Zone] Special Areas of Conservation (Habitats Directive) 2018-10-08 13:56:54.576230
9450 GBR GBR North of Lundy Marine Conservation Zone 2018-10-08 14:04:09.292634
15066 GBR GBR Lundy Marine Conservation Zone 2018-10-08 14:43:41.824497
68808484 GBR GBR !NEW! Lundy [no-take zone] Marine Conservation Zone 2018-10-08 14:44:47.656833
'''

In [311]:
modids = (68808485, 68807800, 68808483, 9450, 15066, 68808484)

def comma_splitter(tag_string):
    return [t.strip() for t in tag_string.split(',') if t.strip()]

In [337]:
import taggit

mpatlas_pgweb = '/Users/russmo/Code/postgres/mpatlas_20181012_webedits.gpkg'
mds = DataSource(mpatlas_pgweb)

allfields = mds[0].fields.copy()
removefields = ['mpa_id', 'categories', ]
fields = [f for f in allfields if f not in removefields]

for feat in mds['mpatlas_20181012_webedits']:
    print(feat['mpa_id'].value, feat['name'].value, feat['designation'].value)
    try:
        m, created = Mpa.objects.get_or_create(name=feat['name'].value, designation=feat['designation'].value, verification_state=feat['verification_state'].value)
        for f in fields:
            setattr(m, f, feat[f].value)
        m.geom = feat.geom.geos
        m.save()
        m.categories.clear()
        m.categories.add(*comma_splitter(feat['categories'].value))
        print(m.pk)
    except(Mpa.DoesNotExist):
        print('  Error')

68808485 Naufrágio Queimado Parque Estadual Marinho
Carto Error for mpa_id 68808741: Access denied
68808741
68808484 Lundy [no-take zone] Marine Conservation Zone
Carto Error for mpa_id 68808742: Access denied
68808742
68808483 Flamborough Head [No-Take Zone] Special Areas of Conservation (Habitats Directive)
Carto Error for mpa_id 68808743: Access denied
68808743
68807800 Flamborough Head Special Areas of Conservation (Habitats Directive)
Carto Error for mpa_id 68807800: Access denied
68807800
15066 Lundy Marine Conservation Zone
Carto Error for mpa_id 15066: Access denied
15066
9450 North of Lundy Marine Conservation Zone
Carto Error for mpa_id 9450: Access denied
9450


In [332]:
import datetime
mm = Mpa.objects.filter(name__icontains='Lundy [no-take zone]')
list(mm.values_list('created_date', 'mpa_id', 'name', 'designation'))
mm.filter(created_date__gte=datetime.datetime(2018,10,12))

<QuerySet []>